In [1]:
import geeViz.getImagesLib as gil
import geeViz.geePalettes as palette
import geeViz.assetManagerLib as aml
import math,os
ee = gil.ee
Map = gil.Map

Initializing GEE
Cached project id file path: C:\Users\ihousman\.config\earthengine\credentials.proj_id
Cached project id: lcms-292214
Successfully initialized
geeViz package folder: c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz


In [11]:

Map.clearMap()
Map.port = 1234
# Set projection (for visualizing, must reproject terrain-based outputs)
output_collection = 'projects/lcms-292214/assets/Ancillary/Wetland/Height_Above_Channel'
aml.create_asset(output_collection,ee.data.ASSET_TYPE_IMAGE_COLL)


# Get NHD flowlines and waterbodies
# Filter out as needed: https://www.usgs.gov/ngp-standards-and-specifications/national-hydrography-dataset-nhd-data-dictionary-feature-classes
nhd_files  = [f['id'] for f in ee.data.getList({'id':"projects/sat-io/open-datasets/NHD"})]

# flowlines = ee.FeatureCollection([ee.FeatureCollection(f+'/NHDFlowline') for f in nhd_files]).flatten()
# waterbodies = ee.FeatureCollection([ee.FeatureCollection(f+'/NHDWaterbody') for f in nhd_files]).flatten()
states = ee.FeatureCollection("TIGER/2018/States")
print(states.first().toDictionary().getInfo())
def getHAC(flowlines,waterbodies,nm,crs,transform):
    flowlines = flowlines.filter(ee.Filter.eq('ftype',460))
    
    waterbodies = waterbodies.filter(ee.Filter.eq('ftype',390))

    # Merge NHD and set a dummy property for rasterizing
    nhd= flowlines.merge(waterbodies)
    nhd = nhd.map(lambda f:f.set('prop',1))
    Map.addLayer(nhd,{'styleParams':{'lineType':'dashed','color':'80F','fillColor':'0FF3'}},f'{nm}- NHD Flowlines and Waterbodies',False)


    # Rasterize NHD
    nhd_rast = nhd.reduceToImage(['prop'],ee.Reducer.first())
    Map.addLayer(nhd_rast,{'min':1,'max':1,'palette':'00D'},f'{nm}- NHD Rast',False)

    # Get distance raster (just for visualizing)
    dist = nhd_rast.distance(ee.Kernel.euclidean(1000,'meters'),False)
    Map.addLayer(dist.reproject(crs,transform),{'min':0,'max':1000,'palette':palette.matplotlib['magma'][7]},f'{nm}-Distance From NHD (meters)',False)

    # Bring in terrain data
    dem = ee.Image("USGS/3DEP/10m").resample('bicubic')#.focalMean(3.5,'circle','pixels')

    # Compute slope in radians
    slope =ee.Terrain.slope(dem).multiply(math.pi).divide(180)
    Map.addLayer(slope.reproject(crs,transform),{'min':0,'max':0.4,'palette':palette.matplotlib['inferno'][7]},f'{nm}-Slope (Radians)',False)


    # Compute the height above NHD 
    hac = slope.tan().cumulativeCost(nhd_rast,1500).rename(['Meters_Above_Channel']).int16()
    Map.addLayer(hac.reproject(crs,transform),{'min':0,'max':80,'palette':palette.matplotlib['plasma'][7]},f'{nm}-Height Above Channel (meters)',True)

    # Mask for valley bottom
    vb = hac.lte(3).selfMask().rename('Valley_Bottom').set({'Valley_Bottom_class_names':['Valley Bottom'],'Valley_Bottom_class_values':[1],'Valley_Bottom_class_palette':['0088FF']})
    Map.addLayer(vb.reproject(crs,transform),{'autoViz':True}, f'{nm}-Valley Bottom',True)
    return hac


for nhd_file in nhd_files[1:3]:
    # print(nhd_file)
    nm = os.path.basename(nhd_file).split('NHD_')[1]
    state = states.filter(ee.Filter.eq('STUSPS',nm)).first()
    print(nm,state.get('STUSPS').getInfo())
    crs = gil.common_projections['NLCD_CONUS']['crs']
    transform = gil.common_projections['NLCD_CONUS']['transform']

    if os.path.basename(nhd_file).find('_AK')> -1:
        crs = gil.common_projections['NLCD_AK']['crs']
        transform = gil.common_projections['NLCD_AK']['transform']
    elif os.path.basename(nhd_file).find('_HI')> -1:
        crs = gil.common_projections['NLCD_HI']['crs']
        transform = gil.common_projections['NLCD_HI']['transform']
    res = 10
    transform[0] = res
    transform[4] = -res

    flowlines = ee.FeatureCollection(nhd_file+'/NHDFlowline')
    waterbodies = ee.FeatureCollection(nhd_file+'/NHDWaterbody') 
    hac = getHAC(flowlines,waterbodies,nm,crs,transform)

    output_name = f'{nm}_Height_Above_NHD_Flowline_and_Waterbody_{res}m'
    output_asset = f'{output_collection}/{output_name}'
    gil.exportToAssetWrapper(hac,output_name,output_asset,roi =state,crs = crs,transform = transform)
Map.turnOnInspector()
Map.setQueryTransform(transform)
# Map.view()


Found the following sub directories:  ['Ancillary', 'Wetland', 'Height_Above_Channel']
Will attempt to create them if they do not exist
Asset projects/lcms-292214/assets/Ancillary already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland/Height_Above_Channel already exists
{'ALAND': 348021896, 'AWATER': 1550236201, 'DIVISION': '0', 'FUNCSTAT': 'A', 'GEOID': '78', 'INTPTLAT': '+18.3267480', 'INTPTLON': '-064.9712508', 'LSAD': '00', 'MTFCC': 'G4000', 'NAME': 'United States Virgin Islands', 'REGION': '9', 'STATEFP': '78', 'STATENS': '01802710', 'STUSPS': 'VI'}
AL AL
Adding layer: AL- NHD Flowlines and Waterbodies
Adding layer: AL- NHD Rast
Adding layer: AL-Distance From NHD (meters)
Adding layer: AL-Slope (Radians)
Adding layer: AL-Height Above Channel (meters)
Adding layer: AL-Valley Bottom
AL_Height_Above_NHD_Flowline_and_Waterbody_10m currently exists or is being exported and overwrite = False. Set overwite = T